In [11]:
import getpass
import os

tavily_api_key = os.environ.get('TAVILY_API_KEY', '')
api_key = os.environ.get('OPENAI_API_KEY', '')

if not api_key:
    print('You need to set the OPENAI_API_KEY environment variable to use this script.')
if not tavily_api_key:
    print('You need to set the TAVILY_API_KEY environment variable to use this script.')

In [19]:
llm_model = "gpt-4-0125-preview"

# Built in retriever

In [125]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults()

In [153]:
tavily_tool.invoke("how do you get a multiball in medival madness?")

[{'url': 'http://www.pinball.org/rules/medievalmadness.html',
  'content': 'Castle Crusher\nTo get the "Castle Crusher" light, you must destroy the 6 Castles of the King\nof Payne and All The King\'s Men: Sir Howard Hurtz (a mobster), Francois de\nGrimm (a Frenchman), the Earl of Ego (a narcissist), the Duke of Bourbon (a\ndrunk), and Sir Psycho (a... uh, psycho).\n The third shot allows you to play the brief "The Catapult" game, where you\ncan choose which of 5 items to throw at the castle -- a duck, a cat, a cow, a\nbowling ball, and a skull head (not that Skull head, sorry, but sounds like\nit).\n If you have Web access, you may want to direct your browser to the Wms web\nsite at http://www.pinball.wms.com/games/medieval -- the site has a superb\n"Explore the Playfield" feature, allowing you to see the MM playfield and\nthe function of all major shots.\n Although it is\nrelatively easy to hit the Trolls anyway, you may want to keep this in mind\nif you have 2 seconds left on the clo

# Accessing information from reddit

In [126]:
from langchain_community.document_loaders import RedditPostsLoader

reddit_loader = RedditPostsLoader(
    client_id="dBUWlJtE3mfzcruyAY1j4Q",
    client_secret="fRFhEWtFLmT6Hmhrn6dln3DpHQUdXA",
    user_agent="extractor by u/Master_Ocelot8179",
    categories=["new", "hot"],  # List of categories to load posts from
    mode="subreddit",
    search_queries=[
        "pinball",
    ],  # List of subreddits to load posts from
    number_posts=100,  # Default value is 10
)

In [149]:
docs = reddit_loader.load()
docs

[Document(page_content='about 2 year old Star Wars.  Several things are not working including the rail gun function. How do I know if im getting ripped off by service?', metadata={'post_subreddit': 'r/pinball', 'post_category': 'new', 'post_title': 'Stern pinball owner. How do I find someone to fix/service it?', 'post_score': 2, 'post_id': '1bitc3c', 'post_url': 'https://www.reddit.com/r/pinball/comments/1bitc3c/stern_pinball_owner_how_do_i_find_someone_to/', 'post_author': Redditor(name='liberty4u2')}),
 Document(page_content='I have just got this addams family pinball cabinet, it has this validador model but i cannot figure out which coin or token I should use on it.\nFor now leaving it at free play...\n\nDoes anyone have any info or know somewhere i can figure this out ?', metadata={'post_subreddit': 'r/pinball', 'post_category': 'new', 'post_title': 'Coin acceptor /validador model AD 81 P 10', 'post_score': 5, 'post_id': '1birosh', 'post_url': 'https://i.redd.it/g1kq7a8o5cpc1.jpeg'

# Building a custom Retriever

In [127]:
from bs4 import BeautifulSoup as Soup
import requests
def get_sub_urls(url):
    html_content = requests.get(url).text

    soup = Soup(html_content, "html.parser")
    links = soup.find_all('a')  # Find all anchor tags
    urls = [link.get('href') for link in links if link.get('href') is not None]
    # Add http://www.pinball.org/rules/ to all urls without http
    urls = ["" if url.startswith('http') else 'http://www.pinball.org/rules/' + url for url in urls]
    # Remove urls that are not valid
    urls = [url for url in urls if url.startswith('http://www.pinball.org/rules/')]

    return urls


In [128]:
pinball_url = "http://www.pinball.org/rules/index-old.html"
pinball_sub_urls = get_sub_urls(pinball_url)

In [129]:

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup as Soup
import nest_asyncio

nest_asyncio.apply()


loader = WebBaseLoader(
    pinball_sub_urls,
    continue_on_failure=True
)


loader.requests_per_second = 2
docs = loader.aload()


Fetching pages:  77%|#######7  | 145/188 [00:25<00:07,  5.84it/s]Error fetching http://www.pinball.org/rules/starwarsepisodei2.html, skipping due to continue_on_failure=True
Error fetching http://www.pinball.org/rules/starwarstrilogy2.html, skipping due to continue_on_failure=True
Fetching pages: 100%|##########| 188/188 [00:33<00:00,  5.58it/s]


In [135]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

vectore_store_index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=OpenAIEmbeddings(api_key=api_key),
).from_loaders([loader, reddit_loader])

In [15]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings(api_key=api_key))


In [142]:
#retriever = vector.as_retriever()
retriever = vectore_store_index.vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x29ccb0b80>)

In [171]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "pinball_search",
    "Search for information about Pinball. For any questions about Pinball, you must use this tool! Do not combine multiple sources of information to produce answers. If there are multiple sources that provide answers you should prompt back to the user and ask which one they want to use.",
)

In [172]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=llm_model, temperature=0,api_key=api_key)

In [173]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain, SimpleSequentialChain

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a pinball assistant, always giving as short and concise answers as possible to pinball-related questions. You do not know anything else but pinball. You can use the `pinball_search` tool to find information about pinball.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [174]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents import create_openai_functions_agent
tools = [tavily_tool, retriever_tool]
llm_with_tools = llm.bind_tools(tools)


from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

#agent = create_openai_functions_agent(llm, tools, prompt)
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [175]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [176]:

agent_executor.invoke({"input": "Hello"})



> Entering new AgentExecutor chain...
Hello! How can I assist you with pinball today?

> Finished chain.


{'input': 'Hello', 'output': 'Hello! How can I assist you with pinball today?'}

In [179]:
retriever.get_relevant_documents("how do you get a multiball in medival madness?")

[Document(page_content="Multiball\nMultiball can be started in two ways.  Locking two balls in the Main\nHabitrail will start 3-ball multiball when you launch the third ball.  The\nother way is to start Cave-In or Jaws of Life during your first run through the\nmodes.  Getting multiball this way will start a 2-ball multiball if there is\nnot a ball locked in the Main Habitrail, and 3-ball if there is a ball locked\nthere.  (Note, you don't actually have to play Cave-In or Jaws of Life.  See\nthree paragraphs down for more info.)\n\nThe object of Multiball is to collect jackpots.  The first jackpot can be\ncollected either at the Helipad or the Emergency Room.  After that, only the\nEmergency Room awards jackpots; the Helipad increases the jackpot value.  The\njackpot starts at 10M.  Each shot to the Helipad to increase the jackpot adds\n30M, and collecting a jackpot makes the next jackpot double the value of the\none just collected.  The jackpot value maxes at 300M.", metadata={'source

In [177]:
#debug
import langchain
langchain.debug = False
agent_executor.invoke({"input": "What is the best strategy in Indiana Jones"})



> Entering new AgentExecutor chain...

Invoking: `pinball_search` with `{'query': 'Indiana Jones pinball strategy'}`


Pinball Archive Rule Sheet: Indiana Jones







  Indiana Jones









Adrian's Guide to
Indiana Jones the Pinball Adventure

Version 2.3


This rulesheet has been compiled and is maintained by me (Adrian)
with a substantial amount of input from a lot of r.g.p readers and
is kept on the pinball archive at rahul.net.  If you find anything
that is incorrect or missing, mail me and I'll make an effort to
include it.

Disclaimer: The usual; Indiana Jones the Pinball Adventure is surely a
a trademark of Williams Electronic Games Inc., and is
licensed from Lucasfilm.  Multi-Ball is also a Williams
trademark.

Preliminary Notes:

A third strategy is to simply fire the ball at the Left and Right Lanes
whenever possible, which keeps the ball away from the bottom half of the
playfield and thus relatively safe.  Remember, though, that the Upper Jet
Bumpers will send the ball

{'input': 'What is the best strategy in Indiana Jones',
 'output': 'For Indiana Jones the Pinball Adventure, here are some strategies:\n\n1. **Left and Right Lanes Strategy**: Aim for the Left and Right Lanes to keep the ball in the upper half of the playfield, which is relatively safer. However, this strategy offers little reward since Bonus Multipliers are not very valuable without a high Bonus score. Eventually, you might need to aim for Drop Targets for quick points.\n\n2. **Battle Field Strategy**: Similar to the Path Of Adventure, the Battle Field is located in the same place but is more square. It features yellow drop targets and stand-up targets. Control a bat with the flippers to hit the ball and complete objectives. Success leads the ball to the upper flipper, while failure might get the ball stuck.\n\n3. **Arrow Shots and Jackpot Shots**: Focus on major Arrow Shots (Left Orbit, Left Ramp, Battle Field, Wall, Right Ramp, Start Scene, Right Orbit, Side Loop) and Jackpot Shots 